In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import re
import os

os.chdir('..')
os.chdir('results')

In [6]:
def get_path(dataset='FordA', model_name='ResidualCNN', reg=True, exp_name=None):
    attack_type = 'fgsm_disc_attack' if reg else 'fgsm_attack'
    exp = '_' + exp_name if exp_name else ''
    
    path = '/'.join([dataset, model_name, 'attack', attack_type + exp])

    return path



def combined_df(dataset='FordA', model_name='ResidualCNN', reg=True, exp_name=None):

    path = get_path(dataset=dataset, model_name=model_name, reg=reg, exp_name=exp_name)
    
    files = os.listdir(path)
    files = [f for f in files if f.endswith('.csv')]

    pattern = r"(?<==).*?(?=.csv)"
    id_pattern = fr"(?<={dataset}_).*?(?=_alpha)"

    all_res = pd.DataFrame()

    for file in files:
        curr_res = pd.read_csv(path + '/' + file, index_col=0)

        curr_res['alpha'] = float(re.search(pattern, file)[0])
        curr_res['model_id'] = re.search(id_pattern, file)[0]

        curr_res.columns = [column.strip(' ') for column in curr_res.columns] 

        all_res = pd.concat([all_res, curr_res], axis=0)

    return all_res.reset_index(drop=True)



def aggregated_results(
    dataset='FordA', 
    model_name='ResidualCNN', 
    exp_name='',
    reg=True,  
    metr='F_EFF_CONC',
    metr_max=True,
    k=1,
    vis = False,
    figsize=(20, 30),
    save_pic=False):

    all_df = combined_df(dataset=dataset, model_name=model_name, reg=reg, exp_name=exp_name)
    # print(all_df)

    grouped = all_df.groupby(['eps', 'alpha'])
    result = grouped.apply(lambda x: x.nlargest(k, metr, keep='last') if metr_max else x.nsmallest(k, metr, keep='last'))


    if vis:
        
        fig, ax = plt.subplots(result['alpha'].nunique(), result['eps'].nunique(), figsize=figsize, sharey=True)
        fig.suptitle(f'{dataset}_{model_name}')

        all_alps = result['alpha'].unique()
        all_eps = result['eps'].unique()

        path = get_path(dataset=dataset, model_name=model_name, reg=reg, exp_name=exp_name)

        for i, row in result.iterrows():

            alp = int(row['alpha']) if row['alpha'].is_integer() else row['alpha']
            eps = row['eps']

            file_path = path + '/' + f'aa_res_{dataset}_{int(row["model_id"])}_alpha={alp}.csv'

            curr_df = pd.read_csv(file_path, index_col=0)
            curr_df.columns = [column.strip(' ') for column in curr_df.columns]

            curr_df = curr_df[curr_df['eps']==eps].set_index('step_id')

            eps_id = np.where(all_eps==eps)[0][0]
            alp_id = np.where(all_alps==alp)[0][0]

            ax[alp_id, eps_id].plot(curr_df[['ACC', 'PROB_HID']])
            ax[alp_id, eps_id].axvline(int(row["step_id"]), ls='--', alpha=0.2, color='black')

            ax[alp_id, eps_id].set_title(f'eps={eps}, alpha={alp}, {metr}={row[metr]}, step={int(row["step_id"])}')
            ax[alp_id, eps_id].legend(['ACC', 'PROB_HID'])
        
        fig.tight_layout()
        fig.subplots_adjust(top=0.95)

        if save_pic:
            plt.savefig(f'../{dataset}_{model_name}.jpeg', dpi=120)

    return result

In [7]:
combined_df(dataset='Coffee', model_name='RNNA', exp_name="MODE").info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1836 entries, 0 to 1835
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ACC             1836 non-null   float64
 1   ROC             1836 non-null   float64
 2   PR              1836 non-null   float64
 3   F1              1836 non-null   float64
 4   EFF             1836 non-null   float64
 5   L1              1836 non-null   float64
 6   ACC_CORRECT     1836 non-null   float64
 7   ACC_ORIG_ADV    1836 non-null   float64
 8   ROUGHNESS       1836 non-null   float64
 9   ROUGHNESS_NORM  1836 non-null   float64
 10  PROB_HID        1836 non-null   float64
 11  ACC_DISC        1836 non-null   float64
 12  F1_DISC         1836 non-null   float64
 13  ROC_AUC_DISC    1836 non-null   float64
 14  CONC            1836 non-null   float64
 15  F_EFF_CONC      1818 non-null   float64
 16  step_id         1836 non-null   float64
 17  eps             1836 non-null   f

In [12]:
res = pd.DataFrame()

for model in ['ResidualCNN', 'RNNA', 'TST', 'S4', 'PatchTST', 'TS2VEC']:
    for ds in ['GunPoint', 'PowerCons', 'Wafer', 'Coffee']:
        try:
            aa = aggregated_results(dataset=ds, model_name=model, exp_name="MODE")
            aa['model'] = model
            aa['dataset'] = ds

            res = pd.concat([res, aa], axis=0)
        except:
            print(ds, model)

Wafer S4
Wafer PatchTST
Wafer TS2VEC


In [17]:
res.to_csv('Table_res')

In [13]:
res.groupby(['dataset', 'model']).max('F_EFF_CONC').reset_index().pivot(index='model', columns='dataset', values='F_EFF_CONC')

dataset,Coffee,GunPoint,PowerCons,Wafer
model,,,,
PatchTST,1.0,0.8248,0.779100,NaN
RNNA,1.0,0.4347,0.252500,0.1039
ResidualCNN,1.0,0.4716,0.948376,0.7080
S4,1.0,1.0000,0.397600,NaN
TS2VEC,1.0,0.7573,0.913800,NaN
TST,1.0,0.3554,0.926621,0.5989
